The goal of this notebook is to summarize the best models from each algo, and save to evalute other datasets

In [1]:
import numpy as np
import pandas as pd
from time import gmtime, strftime, time
import pickle

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, ShuffleSplit, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler


from sklearn.metrics import make_scorer, mean_squared_error, accuracy_score 

from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

import keras
from keras import models
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.wrappers.scikit_learn import KerasRegressor

import matplotlib.pyplot as plt
import seaborn as sns

from Extract_Data import extract_data

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Before running the extract_data function, don't forget to update the data with the latest at finance.yahoo.com

In [21]:
aapl_pca, PCA_df, df = extract_data('AAPL', None)
PCA_df_rand = shuffle(PCA_df, random_state = 0)
df_rand = shuffle(df, random_state = 0)

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError

Getting the datasets ready, separating out the features and labels. Please note that we have 4 datasets:
1. PCA
2. PCA with random order
3. regular dataframe
4. regular dataframe with random order

In [22]:
PCA_rand_X = PCA_df_rand[['Dimension 1', 'Dimension 2', 'Dimension 3', 'Dimension 4',
                          'Dimension 5', 'Dimension 6', 'Dimension 7']]
PCA_rand_y = PCA_df_rand[['Adj Close 1day', 'Adj Close 5day',
                          'Adj Close 1day pct_change', 'Adj Close 5day pct_change',
                          'Adj Close 1day pct_change cls', 'Adj Close 5day pct_change cls']]

PCA_X = PCA_df[['Dimension 1', 'Dimension 2', 'Dimension 3', 'Dimension 4',
                'Dimension 5', 'Dimension 6', 'Dimension 7']]
PCA_y = PCA_df[['Adj Close 1day', 'Adj Close 5day',
                'Adj Close 1day pct_change', 'Adj Close 5day pct_change',
                'Adj Close 1day pct_change cls', 'Adj Close 5day pct_change cls']]

df_rand_X = df_rand[['Open', 'High', 'Low', 'Adj Close', 'Volume', 'Range', 'MA5 Adj Close', 'MA5 Volume',
                     'MA5 Adj Close pct_change', 'MA5 Volume pct_change']]
df_rand_y = df_rand[['Adj Close 1day', 'Adj Close 5day', 'Adj Close 1day pct_change',
                     'Adj Close 5day pct_change', 'Adj Close 1day pct_change cls',
                     'Adj Close 5day pct_change cls']]

df_X = df[['Open', 'High', 'Low', 'Adj Close', 'Volume', 'Range', 'MA5 Adj Close', 'MA5 Volume',
           'MA5 Adj Close pct_change', 'MA5 Volume pct_change']]
df_y = df[['Adj Close 1day', 'Adj Close 5day', 'Adj Close 1day pct_change',
           'Adj Close 5day pct_change', 'Adj Close 1day pct_change cls',
           'Adj Close 5day pct_change cls']]

Let's take a quick look of the datasets

In [23]:
#split into train, test, validation sets
PCA_rand_Xtrain, PCA_rand_Xtest, PCA_rand_ytrain, PCA_rand_ytest = train_test_split(PCA_rand_X, PCA_rand_y, test_size = 0.2)

n_split = int(len(df_y) * 0.8)
PCA_Xtrain, PCA_ytrain = np.array(PCA_X)[:n_split, :], np.array(PCA_y)[:n_split] 
PCA_Xtest, PCA_ytest = np.array(PCA_X)[n_split:, :], np.array(PCA_y)[n_split:]

df_rand_Xtrain, df_rand_Xtest, df_rand_ytrain, df_rand_ytest = train_test_split(df_rand_X, df_rand_y, test_size = 0.2)

df_Xtrain, df_ytrain = np.array(df_X)[:n_split, :], np.array(df_y)[:n_split] 
df_Xtest, df_ytest = np.array(df_X)[n_split:, :], np.array(df_y)[n_split:]

In [24]:
print(PCA_rand_Xtrain.shape, PCA_rand_Xtest.shape, PCA_rand_ytrain.shape, PCA_rand_ytest.shape)
print(PCA_Xtrain.shape, PCA_Xtest.shape, PCA_ytrain.shape, PCA_ytest.shape)

print(df_rand_Xtrain.shape, df_rand_Xtest.shape, df_rand_ytrain.shape, df_rand_ytest.shape)
print(df_Xtrain.shape, df_Xtest.shape, df_ytrain.shape, df_ytest.shape)

(5770, 7) (1443, 7) (5770, 6) (1443, 6)
(5770, 7) (1443, 7) (5770, 6) (1443, 6)
(5770, 10) (1443, 10) (5770, 6) (1443, 6)
(5770, 10) (1443, 10) (5770, 6) (1443, 6)


My target variables:
- 0: 'Adj Close 1day'
- 1: 'Adj Close 5day'
- 2: 'Adj Close 1day pct_change'
- 3: 'Adj Close 5day pct_change'
- 4: 'Adj Close 1day pct_change cls'
- 5: 'Adj Close 5day pct_change cls'

### Best performing:
#### Adj Close 1day: RandomForestRegressor Default with Original Randomized

#### Adj Close 5day: RandomForestRegressor Default with Original Randomized

#### Adj Close 1day pct_change: LSTM Optimized with Original
layer1 = 512, layer2 = 1024, epochs = 100, batch_size = 500

#### Adj Close 5day pct_change: RandomForestRegressor Default with PCA Randomized

#### Adj Close 1day pct_change cls: XGBoost Optimized with PCA Randomized
XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
        gamma=0, learning_rate=0.001, max_delta_step=0, max_depth=3,
        min_child_weight=1, missing=None, n_estimators=700, nthread=-1,
        objective='binary:logistic', reg_alpha=0.1, reg_lambda=1,
        scale_pos_weight=1, seed=0, silent=True, subsample=1)

#### Adj Close 5day pct_change cls: XGBoost Optimized with Original
XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
        gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=8,
        min_child_weight=1, missing=None, n_estimators=600, nthread=-1,
        objective='binary:logistic', reg_alpha=0.01, reg_lambda=1,
        scale_pos_weight=1, seed=0, silent=True, subsample=1)

Train Model and Save
#### Adj Close 1day: RandomForestRegressor Default with Original Randomized

In [26]:
original_perf = dict()

In [27]:
aapl_0_model = RandomForestRegressor()
aapl_0_model.fit(df_rand_Xtrain, df_rand_ytrain['Adj Close 1day'])

train_error = mean_squared_error(aapl_0_model.predict(df_rand_Xtrain), df_rand_ytrain['Adj Close 1day'])/ np.mean(df_rand_ytest['Adj Close 1day'])
test_error = mean_squared_error(aapl_0_model.predict(df_rand_Xtest), df_rand_ytest['Adj Close 1day'])/ np.mean(df_rand_ytest['Adj Close 1day'])

print('Train error: {}% of mean'.format(train_error * 100))
print('Test error: {}% of mean'.format(test_error * 100))

original_perf['Adj Close 1day'] = [train_error, test_error]

Train error: 0.003739574414007908% of mean
Test error: 0.019565380072076866% of mean


In [20]:
aapl_0_model_pkl_filename = 'Models/aapl_0_model.pkl'
with open(aapl_0_model_pkl_filename, 'wb') as file:  
    pickle.dump(aapl_0_model, file)

#### Adj Close 5day: RandomForestRegressor Default with Original Randomized

In [28]:
aapl_1_model = RandomForestRegressor()
aapl_1_model.fit(df_rand_Xtrain, df_rand_ytrain['Adj Close 5day'])

train_error = mean_squared_error(aapl_1_model.predict(df_rand_Xtrain), df_rand_ytrain['Adj Close 5day']) / np.mean(df_rand_ytrain['Adj Close 5day'])
test_error = mean_squared_error(aapl_1_model.predict(df_rand_Xtest), df_rand_ytest['Adj Close 5day']) / np.mean(df_rand_ytest['Adj Close 5day'])

print('Train error: {}% of mean'.format(train_error * 100))
print('Test error: {}% of mean'.format(test_error * 100))

original_perf['Adj Close 5day'] = [train_error, test_error]

Train error: 0.01251256137710871% of mean
Test error: 0.075513349941878% of mean


In [22]:
aapl_1_model_pkl_filename = 'Models/aapl_1_model.pkl'
with open(aapl_1_model_pkl_filename, 'wb') as file:  
    pickle.dump(aapl_1_model, file)

#### Adj Close 1day pct_change: LSTM Optimized with Original
layer1 = 512, layer2 = 1024, epochs = 100, batch_size = 500

In [35]:
def window_transform_series(X, y, window_size):
    # containers for input/output pairs
    X_result = []
    y_result = []
    #print(series)
    #print(window_size)
    for i in range(len(y) - window_size):
        X_result.append(X[i: i + window_size])
        y_result.append(y[i + window_size])
        #print(i)
        #print(series[i: i + window_size])
        #print(series[i + window_size])
    # reshape each

    print(X_result[:3])
    print(np.asarray(X_result)[:3])
    wait = input('wait')

    X_result = np.asarray(X_result)
    X_result.shape = (np.shape(X_result)[0:3])

    y_result = np.asarray(y_result)
    y_result.shape = (len(y_result), 1)

    return X_result, y_result

In [24]:
target_variable = 2
window_size = 5
batch_size = 500

Xtrain_LSTM, ytrain_LSTM = window_transform_series(df_Xtrain, df_ytrain[:, target_variable], window_size = window_size)
Xtest_LSTM, ytest_LSTM = window_transform_series(df_Xtest, df_ytest[:, target_variable], window_size = window_size)


good_size_train = Xtrain_LSTM.shape[0] - Xtrain_LSTM.shape[0] % batch_size
Xtrain_LSTM = Xtrain_LSTM[-good_size_train:]
ytrain_LSTM = ytrain_LSTM[-good_size_train:]

good_size_test = Xtest_LSTM.shape[0] - Xtest_LSTM.shape[0] % batch_size
Xtest_LSTM = Xtest_LSTM[-good_size_test:]
ytest_LSTM  = ytest_LSTM [-good_size_test:]


aapl_2_model = Sequential()
aapl_2_model.add(LSTM(512, batch_input_shape = (batch_size, Xtrain_LSTM.shape[1], Xtrain_LSTM.shape[2]),
               stateful = True, return_sequences = True))
aapl_2_model.add(LSTM(1024))
aapl_2_model.add(Dense(1))
aapl_2_model.compile(loss = 'mean_squared_error', optimizer = 'adam')
aapl_2_model.fit(Xtrain_LSTM, ytrain_LSTM, epochs = 100, batch_size = batch_size, verbose = 1, shuffle = False)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/100
5500/5500 [==============================] - 22s - loss: 4.8511    
Epoch 2/100
5500/5500 [==============================] - 19s - loss: 0.0706    
Epoch 3/100
5500/5500 [==============================] - 19s - loss: 0.0172    
Epoch 4/100
5500/5500 [==============================] - 19s - loss: 0.0060    
Epoch 5/100
5500/5500 [==============================] - 19s - loss: 0.0030    
Epoch 6/100
5500/5500 [==============================] - 19s - loss: 0.0016    
Epoch 7/100
5500/5500 [==============================] - 19s - loss: 0.0011    
Epoch 8/100
5500/5500 [==============================] - 19s - loss: 0.0012    
Epoch 9/100
5500/5500 [==============================] - 19s - loss: 0.0011    
Epoch 10/100
5500/5500 [==============================] - 19s - loss: 0.0011    
Epoch 11/100
5500/5500 [=============================

In [29]:
train_error = mean_squared_error(aapl_2_model.predict(Xtrain_LSTM, batch_size = batch_size), ytrain_LSTM) / np.mean(ytrain_LSTM)
test_error = mean_squared_error(aapl_2_model.predict(Xtest_LSTM, batch_size = batch_size), ytest_LSTM) / np.mean(ytest_LSTM)

print('Train error: {}% of mean'.format(train_error * 100))
print('Test error: {}% of mean'.format(test_error * 100))

original_perf['Adj Close 1day pct_change'] = [train_error, test_error]

Train error: 65.4766642018951% of mean
Test error: 22.135985109734282% of mean


In [29]:
aapl_2_model_pkl_filename = 'Models/aapl_2_model.h5'
aapl_2_model.save(aapl_2_model_pkl_filename)

#### Adj Close 5day pct_change: RandomForestRegressor Default with PCA Randomized

In [30]:
aapl_3_model = RandomForestRegressor()
aapl_3_model.fit(PCA_rand_Xtrain, PCA_rand_ytrain['Adj Close 5day pct_change'])

train_error = mean_squared_error(aapl_3_model.predict(PCA_rand_Xtrain), PCA_rand_ytrain['Adj Close 5day pct_change']) / np.mean(PCA_rand_ytrain['Adj Close 5day pct_change'])
test_error = mean_squared_error(aapl_3_model.predict(PCA_rand_Xtest), PCA_rand_ytest['Adj Close 5day pct_change']) / np.mean(PCA_rand_ytest['Adj Close 5day pct_change'])

print('Train error: {}% of mean'.format(train_error * 100))
print('Test error: {}% of mean'.format(test_error * 100))

original_perf['Adj Close 5day pct_change'] = [train_error, test_error]

Train error: 11.03879180212689% of mean
Test error: 59.489124947324015% of mean


In [35]:
aapl_3_model_pkl_filename = 'Models/aapl_3_model.pkl'
with open(aapl_3_model_pkl_filename, 'wb') as file:  
    pickle.dump(aapl_3_model, file)

#### Adj Close 1day pct_change cls: XGBoost Optimized with PCA Randomized
XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
        gamma=0, learning_rate=0.001, max_delta_step=0, max_depth=3,
        min_child_weight=1, missing=None, n_estimators=700, nthread=-1,
        objective='binary:logistic', reg_alpha=0.1, reg_lambda=1,
        scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [31]:
aapl_4_model = xgb.XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
                                 gamma=0, learning_rate=0.001, max_delta_step=0, max_depth=3,
                                 min_child_weight=1, missing=None, n_estimators=700, nthread=-1,
                                 objective='binary:logistic', reg_alpha=0.1, reg_lambda=1,
                                 scale_pos_weight=1, seed=0, silent=True, subsample=1)
aapl_4_model.fit(PCA_rand_Xtrain, PCA_rand_ytrain['Adj Close 1day pct_change cls'])

train_error = accuracy_score(aapl_4_model.predict(PCA_rand_Xtrain), PCA_rand_ytrain['Adj Close 1day pct_change cls'])
test_error = accuracy_score(aapl_4_model.predict(PCA_rand_Xtest), PCA_rand_ytest['Adj Close 1day pct_change cls'])

print('Train accuracy: {}%'.format(train_error * 100))
print('Test accuracy: {}%'.format(test_error * 100))

original_perf['Adj Close 1day pct_change cls'] = [train_error, test_error]

Train accuracy: 54.523396880415945%
Test accuracy: 52.18295218295218%


c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [32]:
aapl_4_model_pkl_filename = 'Models/aapl_4_model.pkl'
with open(aapl_4_model_pkl_filename, 'wb') as file:  
    pickle.dump(aapl_4_model, file)

#### Adj Close 5day pct_change cls: XGBoost Optimized with Original
XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
        gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=8,
        min_child_weight=1, missing=None, n_estimators=600, nthread=-1,
        objective='binary:logistic', reg_alpha=0.01, reg_lambda=1,
        scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [33]:
target_variable = 5

aapl_5_model = xgb.XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
                                 gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=8,
                                 min_child_weight=1, missing=None, n_estimators=600, nthread=-1,
                                 objective='binary:logistic', reg_alpha=0.01, reg_lambda=1,
                                 scale_pos_weight=1, seed=0, silent=True, subsample=1)
aapl_5_model.fit(df_Xtrain, df_ytrain[:, target_variable])

train_error = accuracy_score(aapl_5_model.predict(df_Xtrain), df_ytrain[:, target_variable])
test_error = accuracy_score(aapl_5_model.predict(df_Xtest), df_ytest[:, target_variable])

print('Train accuracy: {}%'.format(train_error * 100))
print('Test accuracy: {}%'.format(test_error * 100))

original_perf['Adj Close 5day pct_change cls'] = [train_error, test_error]

Train accuracy: 100.0%
Test accuracy: 54.19265419265419%


c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [34]:
aapl_5_model_pkl_filename = 'Models/aapl_5_model.pkl'
with open(aapl_5_model_pkl_filename, 'wb') as file:  
    pickle.dump(aapl_5_model, file)

In [35]:
aapl_PCA_model_pkl_filename = 'Models/aapl_PCA_model.pkl'
with open(aapl_PCA_model_pkl_filename, 'wb') as file:  
    pickle.dump(pca, file)

In [40]:
pd.DataFrame(original_perf)[['Adj Close 1day','Adj Close 5day','Adj Close 1day pct_change','Adj Close 5day pct_change',
                             'Adj Close 1day pct_change cls','Adj Close 5day pct_change cls']]

,Adj Close 1day,Adj Close 5day,Adj Close 1day pct_change,Adj Close 5day pct_change,Adj Close 1day pct_change cls,Adj Close 5day pct_change cls
0,0.000037,0.000125,0.654767,0.110388,0.545234,1.000000
1,0.000196,0.000755,0.221360,0.594891,0.521830,0.541927


I'm going to try to test my model in two directions:
1. Vertical - This involves in testing the model of the same equity, but with more recent data. Using the original PCA
2. Horizontal - This involves in testing the model of different equity. Using the original PCA.

### Vertical Testing with Updated Data

In [125]:
#loading PCA
aapl_PCA_model_pkl_filename = 'Models/aapl_PCA_model.pkl'
with open(aapl_PCA_model_pkl_filename, 'rb') as file:
    aapl_PCA_model = pickle.load(file)
    
aapl_0_model_pkl_filename = 'Models/aapl_0_model.pkl'
with open(aapl_0_model_pkl_filename, 'rb') as file:  
    aapl_0_model = pickle.load(file)
    
aapl_1_model_pkl_filename = 'Models/aapl_1_model.pkl'
with open(aapl_1_model_pkl_filename, 'rb') as file:  
    aapl_1_model = pickle.load(file)
    
aapl_2_model_pkl_filename = 'Models/aapl_2_model.h5'
aapl_2_model = models.load_model(aapl_2_model_pkl_filename)

aapl_3_model_pkl_filename = 'Models/aapl_3_model.pkl'
with open(aapl_3_model_pkl_filename, 'rb') as file:  
    aapl_3_model = pickle.load(file)
    
aapl_4_model_pkl_filename = 'Models/aapl_4_model.pkl'
with open(aapl_4_model_pkl_filename, 'rb') as file:  
    aapl_4_model = pickle.load(file)
    
aapl_5_model_pkl_filename = 'Models/aapl_5_model.pkl'
with open(aapl_5_model_pkl_filename, 'rb') as file:  
    aapl_5_model = pickle.load(file)

In [131]:
_, AAPL2_PCA_df, AAPL2_df = extract_data('AAPL2', aapl_PCA_model)
AAPL2_PCA_df_rand = shuffle(AAPL2_PCA_df, random_state = 0)
AAPL2_df_rand = shuffle(AAPL2_df, random_state = 0)

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError

In [179]:
AAPL2_PCA_rand_X = AAPL2_PCA_df_rand[['Dimension 1', 'Dimension 2', 'Dimension 3', 'Dimension 4',
                                      'Dimension 5', 'Dimension 6', 'Dimension 7']]
AAPL2_PCA_rand_y = AAPL2_PCA_df_rand[['Adj Close 1day', 'Adj Close 5day',
                                      'Adj Close 1day pct_change', 'Adj Close 5day pct_change',
                                      'Adj Close 1day pct_change cls', 'Adj Close 5day pct_change cls']]

AAPL2_PCA_X = AAPL2_PCA_df[['Dimension 1', 'Dimension 2', 'Dimension 3', 'Dimension 4',
                            'Dimension 5', 'Dimension 6', 'Dimension 7']]
AAPL2_PCA_y = AAPL2_PCA_df[['Adj Close 1day', 'Adj Close 5day',
                            'Adj Close 1day pct_change', 'Adj Close 5day pct_change',
                            'Adj Close 1day pct_change cls', 'Adj Close 5day pct_change cls']]

AAPL2_df_rand_X = AAPL2_df_rand[['Open', 'High', 'Low', 'Adj Close', 'Volume', 'Range', 'MA5 Adj Close', 'MA5 Volume',
                                 'MA5 Adj Close pct_change', 'MA5 Volume pct_change']]
AAPL2_df_rand_y = AAPL2_df_rand[['Adj Close 1day', 'Adj Close 5day', 'Adj Close 1day pct_change',
                                 'Adj Close 5day pct_change', 'Adj Close 1day pct_change cls',
                                 'Adj Close 5day pct_change cls']]

AAPL2_df_X = AAPL2_df[['Open', 'High', 'Low', 'Adj Close', 'Volume', 'Range', 'MA5 Adj Close', 'MA5 Volume',
                       'MA5 Adj Close pct_change', 'MA5 Volume pct_change']]
AAPL2_df_y = AAPL2_df[['Adj Close 1day', 'Adj Close 5day', 'Adj Close 1day pct_change',
                       'Adj Close 5day pct_change', 'Adj Close 1day pct_change cls',
                       'Adj Close 5day pct_change cls']]

In [180]:
print(AAPL2_PCA_rand_X.shape, AAPL2_PCA_rand_y.shape)
print(AAPL2_PCA_X.shape, AAPL2_PCA_y.shape)

print(AAPL2_df_rand_X.shape, AAPL2_df_rand_y.shape)
print(AAPL2_df_X.shape, AAPL2_df_y.shape)

(7220, 7) (7220, 6)
(7220, 7) (7220, 6)
(7220, 10) (7220, 6)
(7220, 10) (7220, 6)


In [181]:
AAPL2_perf = dict()

In [182]:
error = mean_squared_error(aapl_0_model.predict(AAPL2_df_rand_X), AAPL2_df_rand_y['Adj Close 1day']) / np.mean(AAPL2_df_rand_y['Adj Close 1day'])

print('Error: {}% of mean'.format(error * 100))

AAPL2_perf['Adj Close 1day'] = error

Error: 0.00664758317532428% of mean


In [183]:
error = mean_squared_error(aapl_1_model.predict(AAPL2_df_rand_X), AAPL2_df_rand_y['Adj Close 5day']) / np.mean(AAPL2_df_rand_y['Adj Close 5day'])

print('Error: {}% of mean'.format(error * 100))

AAPL2_perf['Adj Close 5day'] = error

Error: 0.02403585177153832% of mean


In [184]:
def window_transform_series(X, y, window_size):
    # containers for input/output pairs
    X_result = []
    y_result = []
    #print(series)
    #print(window_size)
    for i in range(len(y) - window_size):
        X_result.append(X[i: i + window_size])
        y_result.append(y[i + window_size])
        #print(i)
        #print(series[i: i + window_size])
        #print(series[i + window_size])
    # reshape each

    X_result = np.asarray(X_result)
    X_result.shape = (np.shape(X_result)[0:3])

    y_result = np.asarray(y_result)
    y_result.shape = (len(y_result), 1)

    return X_result, y_result

In [185]:
target_variable = 2
window_size = 5
batch_size = 500

AAPL2_X_LSTM, AAPL2_y_LSTM = window_transform_series(np.array(AAPL2_df_X), np.array(AAPL2_df_y['Adj Close 1day pct_change']), window_size = window_size)

good_size_train = AAPL2_X_LSTM.shape[0] - AAPL2_X_LSTM.shape[0] % batch_size
AAPL2_X_LSTM = AAPL2_X_LSTM[-good_size_train:]
AAPL2_y_LSTM = AAPL2_y_LSTM[-good_size_train:]

error = mean_squared_error(aapl_2_model.predict(AAPL2_X_LSTM, batch_size = batch_size), np.array(AAPL2_y_LSTM)) / np.mean(AAPL2_y_LSTM)

print('Error: {}% of mean'.format(error * 100))

AAPL2_perf['Adj Close 1day pct_change'] = error

Error: 56.57366989646386% of mean


In [186]:
error = mean_squared_error(aapl_3_model.predict(AAPL2_PCA_rand_X), AAPL2_PCA_rand_y['Adj Close 5day pct_change']) / np.mean(AAPL2_PCA_rand_y['Adj Close 5day pct_change'])

print('Error: {}% of mean'.format(error * 100))

AAPL2_perf['Adj Close 5day pct_change'] = error

Error: 21.65289785536967% of mean


In [187]:
error = accuracy_score(aapl_4_model.predict(AAPL2_PCA_rand_X), AAPL2_PCA_rand_y['Adj Close 1day pct_change cls'])

print('Error: {}% of mean'.format(error * 100))

AAPL2_perf['Adj Close 1day pct_change cls'] = error

Error: 54.099722991689745% of mean


c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [188]:
error = accuracy_score(aapl_5_model.predict(np.array(AAPL2_df_X)), AAPL2_df_y['Adj Close 5day pct_change cls'])

print('Error: {}% of mean'.format(error * 100))

AAPL2_perf['Adj Close 5day pct_change cls'] = error

Error: 90.803324099723% of mean


c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [189]:
AAPL2_perf

{'Adj Close 1day': 6.64758317532428e-05,
 'Adj Close 1day pct_change': 0.5657366989646386,
 'Adj Close 1day pct_change cls': 0.5409972299168975,
 'Adj Close 5day': 0.0002403585177153832,
 'Adj Close 5day pct_change': 0.2165289785536967,
 'Adj Close 5day pct_change cls': 0.9080332409972299}

|MSE and ACC|Adj Close 1day|Adj Close 5day|Adj Close 1day pct_change|Adj Close 5day pct_change|Adj Close 1day pct_change cls|Adj Close 5day pct_change cls|
|-|-|-|-|-|-||
|AAPL train|0.000037|0.000125|0.654767|0.110388|0.545234|1.000000|
|AAPL test |0.000196|0.000755|0.221360|0.594891|0.521830|0.541927|
|AALP2     |6.64e-05|0.000240|0.608699|0.216528|0.540997|0.908033|

### Horizontal Testing with Updated Data

#### Trying SPY

In [190]:
_, SPY_PCA_df, SPY_df = extract_data('SPY', aapl_PCA_model)
SPY_PCA_df_rand = shuffle(SPY_PCA_df, random_state = 0)
SPY_df_rand = shuffle(SPY_df, random_state = 0)

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError

In [191]:
SPY_PCA_rand_X = SPY_PCA_df_rand[['Dimension 1', 'Dimension 2', 'Dimension 3', 'Dimension 4',
                                  'Dimension 5', 'Dimension 6', 'Dimension 7']]
SPY_PCA_rand_y = SPY_PCA_df_rand[['Adj Close 1day', 'Adj Close 5day',
                                  'Adj Close 1day pct_change', 'Adj Close 5day pct_change',
                                  'Adj Close 1day pct_change cls', 'Adj Close 5day pct_change cls']]
SPY_PCA_X = SPY_PCA_df[['Dimension 1', 'Dimension 2', 'Dimension 3', 'Dimension 4',
                        'Dimension 5', 'Dimension 6', 'Dimension 7']]
SPY_PCA_y = SPY_PCA_df[['Adj Close 1day', 'Adj Close 5day',
                        'Adj Close 1day pct_change', 'Adj Close 5day pct_change',
                        'Adj Close 1day pct_change cls', 'Adj Close 5day pct_change cls']]

SPY_df_rand_X = SPY_df_rand[['Open', 'High', 'Low', 'Adj Close', 'Volume', 'Range', 'MA5 Adj Close', 'MA5 Volume',
                             'MA5 Adj Close pct_change', 'MA5 Volume pct_change']]
SPY_df_rand_y = SPY_df_rand[['Adj Close 1day', 'Adj Close 5day', 'Adj Close 1day pct_change',
                             'Adj Close 5day pct_change', 'Adj Close 1day pct_change cls',
                             'Adj Close 5day pct_change cls']]

SPY_df_X = SPY_df[['Open', 'High', 'Low', 'Adj Close', 'Volume', 'Range', 'MA5 Adj Close', 'MA5 Volume',
                   'MA5 Adj Close pct_change', 'MA5 Volume pct_change']]
SPY_df_y = SPY_df[['Adj Close 1day', 'Adj Close 5day', 'Adj Close 1day pct_change',
                   'Adj Close 5day pct_change', 'Adj Close 1day pct_change cls',
                   'Adj Close 5day pct_change cls']]

In [192]:
print(SPY_PCA_rand_X.shape, SPY_PCA_rand_y.shape)
print(SPY_PCA_X.shape, SPY_PCA_y.shape)

print(SPY_df_rand_X.shape, SPY_df_rand_y.shape)
print(SPY_df_X.shape, SPY_df_y.shape)

(6434, 7) (6434, 6)
(6434, 7) (6434, 6)
(6434, 10) (6434, 6)
(6434, 10) (6434, 6)


In [193]:
SPY_perf = dict()

In [194]:
error = mean_squared_error(aapl_0_model.predict(SPY_df_rand_X), SPY_df_rand_y['Adj Close 1day']) / np.mean(SPY_df_rand_y['Adj Close 1day'])

print('Error: {}% of mean'.format(error * 100))

SPY_perf['Adj Close 1day'] = error

Error: 0.05267990202614801% of mean


In [195]:
error = mean_squared_error(aapl_1_model.predict(SPY_df_rand_X), SPY_df_rand_y['Adj Close 5day']) / np.mean(SPY_df_rand_y['Adj Close 5day'])

print('Error: {}% of mean'.format(error * 100))

SPY_perf['Adj Close 5day'] = error

Error: 0.06936597400936317% of mean


In [196]:
target_variable = 2
window_size = 5
batch_size = 500

SPY_X_LSTM, SPY_y_LSTM = window_transform_series(np.array(SPY_df_X), np.array(SPY_df_y['Adj Close 1day pct_change']), window_size = window_size)

good_size_train = SPY_X_LSTM.shape[0] - SPY_X_LSTM.shape[0] % batch_size
SPY_X_LSTM = SPY_X_LSTM[-good_size_train:]
SPY_y_LSTM = SPY_y_LSTM[-good_size_train:]

error = mean_squared_error(aapl_2_model.predict(SPY_X_LSTM, batch_size = batch_size), np.array(SPY_y_LSTM)) / np.mean(SPY_y_LSTM)

print('Error: {}% of mean'.format(error * 100))

SPY_perf['Adj Close 1day pct_change'] = error

Error: 42.1153307025107% of mean


In [197]:
error = mean_squared_error(aapl_3_model.predict(SPY_PCA_rand_X), SPY_PCA_rand_y['Adj Close 5day pct_change']) / np.mean(SPY_PCA_rand_y['Adj Close 5day pct_change'])

print('Error: {}% of mean'.format(error * 100))

SPY_perf['Adj Close 5day pct_change'] = error

Error: 45.66914214140355% of mean


In [198]:
error = accuracy_score(aapl_4_model.predict(SPY_PCA_rand_X), SPY_PCA_rand_y['Adj Close 1day pct_change cls'])

print('Error: {}% of mean'.format(error * 100))

SPY_perf['Adj Close 1day pct_change cls'] = error

Error: 54.35188063413118% of mean


c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [199]:
error = accuracy_score(aapl_5_model.predict(np.array(SPY_df_X)), SPY_df_y['Adj Close 5day pct_change cls'])

print('Error: {}% of mean'.format(error * 100))

SPY_perf['Adj Close 5day pct_change cls'] = error

Error: 49.79794839912962% of mean


c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [200]:
SPY_perf

{'Adj Close 1day': 0.0005267990202614801,
 'Adj Close 1day pct_change': 0.42115330702510695,
 'Adj Close 1day pct_change cls': 0.5435188063413118,
 'Adj Close 5day': 0.0006936597400936318,
 'Adj Close 5day pct_change': 0.45669142141403546,
 'Adj Close 5day pct_change cls': 0.4979794839912962}

|MSE and ACC|Adj Close 1day|Adj Close 5day|Adj Close 1day pct_change|Adj Close 5day pct_change|Adj Close 1day pct_change cls|Adj Close 5day pct_change cls|
|-|-|-|-|-|-||
|AAPL train|0.000037|0.000125|0.654767|0.110388|0.545234|1.000000|
|AAPL test |0.000196|0.000755|0.221360|0.594891|0.521830|0.541927|
|AALP2     |6.64e-05|0.000240|0.608699|0.216528|0.540997|0.908033|
|SPY       |0.000526|0.000693|0.411881|0.456691|0.543518|0.497979|

Had some problems with BTC as some values turned out to be NaN or inf. So trying other equities instead

#### Trying HD

In [201]:
_, HD_PCA_df, HD_df = extract_data('HD', aapl_PCA_model)
HD_PCA_df_rand = shuffle(HD_PCA_df, random_state = 0)
HD_df_rand = shuffle(HD_df, random_state = 0)

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError

In [202]:
HD_PCA_rand_X = HD_PCA_df_rand[['Dimension 1', 'Dimension 2', 'Dimension 3', 'Dimension 4',
                                'Dimension 5', 'Dimension 6', 'Dimension 7']]
HD_PCA_rand_y = HD_PCA_df_rand[['Adj Close 1day', 'Adj Close 5day',
                                'Adj Close 1day pct_change', 'Adj Close 5day pct_change',
                                'Adj Close 1day pct_change cls', 'Adj Close 5day pct_change cls']]

HD_PCA_X = HD_PCA_df[['Dimension 1', 'Dimension 2', 'Dimension 3', 'Dimension 4',
                      'Dimension 5', 'Dimension 6', 'Dimension 7']]
HD_PCA_y = HD_PCA_df[['Adj Close 1day', 'Adj Close 5day',
                      'Adj Close 1day pct_change', 'Adj Close 5day pct_change',
                      'Adj Close 1day pct_change cls', 'Adj Close 5day pct_change cls']]

HD_df_rand_X = HD_df_rand[['Open', 'High', 'Low', 'Adj Close', 'Volume', 'Range', 'MA5 Adj Close', 'MA5 Volume',
                           'MA5 Adj Close pct_change', 'MA5 Volume pct_change']]
HD_df_rand_y = HD_df_rand[['Adj Close 1day', 'Adj Close 5day', 'Adj Close 1day pct_change',
                           'Adj Close 5day pct_change', 'Adj Close 1day pct_change cls',
                           'Adj Close 5day pct_change cls']]

HD_df_X = HD_df[['Open', 'High', 'Low', 'Adj Close', 'Volume', 'Range', 'MA5 Adj Close', 'MA5 Volume',
                 'MA5 Adj Close pct_change', 'MA5 Volume pct_change']]
HD_df_y = HD_df[['Adj Close 1day', 'Adj Close 5day', 'Adj Close 1day pct_change',
                 'Adj Close 5day pct_change', 'Adj Close 1day pct_change cls',
                 'Adj Close 5day pct_change cls']]

In [203]:
print(HD_PCA_rand_X.shape, HD_PCA_rand_y.shape)
print(HD_PCA_X.shape, HD_PCA_y.shape)

print(HD_df_rand_X.shape, HD_df_rand_y.shape)
print(HD_df_X.shape, HD_df_y.shape)

(7213, 7) (7213, 6)
(7213, 7) (7213, 6)
(7213, 10) (7213, 6)
(7213, 10) (7213, 6)


In [204]:
HD_perf = dict()

In [205]:
error = mean_squared_error(aapl_0_model.predict(HD_df_rand_X), HD_df_rand_y['Adj Close 1day']) / np.mean(HD_df_rand_y['Adj Close 1day'])

print('Error: {}% of mean'.format(error * 100))

HD_perf['Adj Close 1day'] = error

Error: 0.01746526433412856% of mean


In [206]:
error = mean_squared_error(aapl_1_model.predict(HD_df_rand_X), HD_df_rand_y['Adj Close 5day']) / np.mean(HD_df_rand_y['Adj Close 5day'])

print('Error: {}% of mean'.format(error * 100))

HD_perf['Adj Close 5day'] = error

Error: 0.09112008410272147% of mean


In [207]:
target_variable = 2
window_size = 5
batch_size = 500

HD_X_LSTM, HD_y_LSTM = window_transform_series(np.array(HD_df_X), np.array(HD_df_y['Adj Close 1day pct_change']), window_size = window_size)

good_size_train = HD_X_LSTM.shape[0] - HD_X_LSTM.shape[0] % batch_size
HD_X_LSTM = HD_X_LSTM[-good_size_train:]
HD_y_LSTM = HD_y_LSTM[-good_size_train:]

error = mean_squared_error(aapl_2_model.predict(HD_X_LSTM, batch_size = batch_size), np.array(HD_y_LSTM)) / np.mean(HD_y_LSTM)

print('Error: {}% of mean'.format(error * 100))

HD_perf['Adj Close 1day pct_change'] = error

Error: 46.43251321996854% of mean


In [208]:
error = mean_squared_error(aapl_3_model.predict(HD_PCA_rand_X), HD_PCA_rand_y['Adj Close 5day pct_change']) / np.mean(HD_PCA_rand_y['Adj Close 5day pct_change'])

print('Error: {}% of mean'.format(error * 100))

HD_perf['Adj Close 5day pct_change'] = error

Error: 56.714993547759576% of mean


In [209]:
error = accuracy_score(aapl_4_model.predict(HD_PCA_rand_X), HD_PCA_rand_y['Adj Close 1day pct_change cls'])

print('Error: {}% of mean'.format(error * 100))

HD_perf['Adj Close 1day pct_change cls'] = error

Error: 53.140163593511716% of mean


c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [210]:
error = accuracy_score(aapl_5_model.predict(np.array(HD_df_X)), HD_df_y['Adj Close 5day pct_change cls'])

print('Error: {}% of mean'.format(error * 100))

HD_perf['Adj Close 5day pct_change cls'] = error

Error: 50.464439206987386% of mean


c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [211]:
HD_perf

{'Adj Close 1day': 0.0001746526433412856,
 'Adj Close 1day pct_change': 0.4643251321996854,
 'Adj Close 1day pct_change cls': 0.5314016359351171,
 'Adj Close 5day': 0.0009112008410272148,
 'Adj Close 5day pct_change': 0.5671499354775957,
 'Adj Close 5day pct_change cls': 0.5046443920698739}

|MSE and ACC|Adj Close 1day|Adj Close 5day|Adj Close 1day pct_change|Adj Close 5day pct_change|Adj Close 1day pct_change cls|Adj Close 5day pct_change cls|
|-|-|-|-|-|-||
|AAPL train|0.000037|0.000125|0.654767|0.110388|0.545234|1.000000|
|AAPL test |0.000196|0.000755|0.221360|0.594891|0.521830|0.541927|
|AALP2     |6.64e-05|0.000240|0.608699|0.216528|0.540997|0.908033|
|SPY       |0.000526|0.000693|0.411881|0.456691|0.543518|0.497979|
|HD        |0.000174|0.000911|0.457377|0.567149|0.531401|0.504644|

#### Trying VXX

In [212]:
_, VXX_PCA_df, VXX_df = extract_data('VXX', aapl_PCA_model)
VXX_PCA_df_rand = shuffle(VXX_PCA_df, random_state = 0)
VXX_df_rand = shuffle(VXX_df, random_state = 0)

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError

In [213]:
VXX_PCA_rand_X = VXX_PCA_df_rand[['Dimension 1', 'Dimension 2', 'Dimension 3', 'Dimension 4',
                                  'Dimension 5', 'Dimension 6', 'Dimension 7']]
VXX_PCA_rand_y = VXX_PCA_df_rand[['Adj Close 1day', 'Adj Close 5day',
                                  'Adj Close 1day pct_change', 'Adj Close 5day pct_change',
                                  'Adj Close 1day pct_change cls', 'Adj Close 5day pct_change cls']]

VXX_PCA_X = VXX_PCA_df[['Dimension 1', 'Dimension 2', 'Dimension 3', 'Dimension 4',
                        'Dimension 5', 'Dimension 6', 'Dimension 7']]
VXX_PCA_y = VXX_PCA_df[['Adj Close 1day', 'Adj Close 5day',
                        'Adj Close 1day pct_change', 'Adj Close 5day pct_change',
                        'Adj Close 1day pct_change cls', 'Adj Close 5day pct_change cls']]

VXX_df_rand_X = VXX_df_rand[['Open', 'High', 'Low', 'Adj Close', 'Volume', 'Range', 'MA5 Adj Close', 'MA5 Volume',
                             'MA5 Adj Close pct_change', 'MA5 Volume pct_change']]
VXX_df_rand_y = VXX_df_rand[['Adj Close 1day', 'Adj Close 5day', 'Adj Close 1day pct_change',
                             'Adj Close 5day pct_change', 'Adj Close 1day pct_change cls',
                             'Adj Close 5day pct_change cls']]

VXX_df_X = VXX_df[['Open', 'High', 'Low', 'Adj Close', 'Volume', 'Range', 'MA5 Adj Close', 'MA5 Volume',
                   'MA5 Adj Close pct_change', 'MA5 Volume pct_change']]
VXX_df_y = VXX_df[['Adj Close 1day', 'Adj Close 5day', 'Adj Close 1day pct_change',
                   'Adj Close 5day pct_change', 'Adj Close 1day pct_change cls',
                   'Adj Close 5day pct_change cls']]

In [214]:
print(VXX_PCA_rand_X.shape, VXX_PCA_rand_y.shape)
print(VXX_PCA_X.shape, VXX_PCA_y.shape)

print(VXX_df_rand_X.shape, VXX_df_rand_y.shape)
print(VXX_df_X.shape, VXX_df_y.shape)

(2403, 7) (2403, 6)
(2403, 7) (2403, 6)
(2403, 10) (2403, 6)
(2403, 10) (2403, 6)


In [215]:
VXX_perf = dict()

In [216]:
error = mean_squared_error(aapl_0_model.predict(VXX_df_rand_X), VXX_df_rand_y['Adj Close 1day']) / np.mean(VXX_df_rand_y['Adj Close 1day'])

print('Error: {}% of mean'.format(error * 100))

VXX_perf['Adj Close 1day'] = error

Error: 112.87181184373117% of mean


In [217]:
error = mean_squared_error(aapl_1_model.predict(VXX_df_rand_X), VXX_df_rand_y['Adj Close 5day']) / np.mean(VXX_df_rand_y['Adj Close 5day'])

print('Error: {}% of mean'.format(error * 100))

VXX_perf['Adj Close 5day'] = error

Error: 111.01988407412603% of mean


In [218]:
target_variable = 2
window_size = 5
batch_size = 500

VXX_X_LSTM, VXX_y_LSTM = window_transform_series(np.array(VXX_df_X), np.array(VXX_df_y['Adj Close 1day pct_change']), window_size = window_size)

good_size_train = VXX_X_LSTM.shape[0] - VXX_X_LSTM.shape[0] % batch_size
VXX_X_LSTM = VXX_X_LSTM[-good_size_train:]
VXX_y_LSTM = VXX_y_LSTM[-good_size_train:]

error = mean_squared_error(aapl_2_model.predict(VXX_X_LSTM, batch_size = batch_size), np.array(VXX_y_LSTM)) / np.mean(VXX_y_LSTM)

print('Error: {}% of mean'.format(error * 100))

VXX_perf['Adj Close 1day pct_change'] = error

Error: -437.9095644124994% of mean


In [219]:
error = mean_squared_error(aapl_3_model.predict(VXX_PCA_rand_X), VXX_PCA_rand_y['Adj Close 5day pct_change']) / np.mean(VXX_PCA_rand_y['Adj Close 5day pct_change'])

print('Error: {}% of mean'.format(error * 100))

VXX_perf['Adj Close 5day pct_change'] = error

Error: -95.26061997273327% of mean


In [220]:
error = accuracy_score(aapl_4_model.predict(VXX_PCA_rand_X), VXX_PCA_rand_y['Adj Close 1day pct_change cls'])

print('Error: {}% of mean'.format(error * 100))

VXX_perf['Adj Close 1day pct_change cls'] = error

Error: 44.77736163129421% of mean


c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [221]:
error = accuracy_score(aapl_5_model.predict(np.array(VXX_df_X)), VXX_df_y['Adj Close 5day pct_change cls'])

print('Error: {}% of mean'.format(error * 100))

VXX_perf['Adj Close 5day pct_change cls'] = error

Error: 54.43196004993758% of mean


c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [222]:
VXX_perf

{'Adj Close 1day': 1.1287181184373116,
 'Adj Close 1day pct_change': -4.379095644124994,
 'Adj Close 1day pct_change cls': 0.44777361631294216,
 'Adj Close 5day': 1.1101988407412602,
 'Adj Close 5day pct_change': -0.9526061997273326,
 'Adj Close 5day pct_change cls': 0.5443196004993758}

|MSE and ACC|Adj Close 1day|Adj Close 5day|Adj Close 1day pct_change|Adj Close 5day pct_change|Adj Close 1day pct_change cls|Adj Close 5day pct_change cls|
|-|-|-|-|-|-||
|AAPL train|0.000037|0.000125|0.654767|0.110388|0.545234|1.000000|
|AAPL test |0.000196|0.000755|0.221360|0.594891|0.521830|0.541927|
|AALP2     |6.64e-05|0.000240|0.608699|0.216528|0.540997|0.908033|
|SPY       |0.000526|0.000693|0.411881|0.456691|0.543518|0.497979|
|HD        |0.000174|0.000911|0.457377|0.567149|0.531401|0.504644|
|VXX       |1.128718|1.110198|-4.374290|-0.952606|0.447773|0.544319|

Seems like the model only fit AAPL train, as it has the lowest MSE and highest accuarcy. The second place is AAPL2. 